In [2]:
import tensorflow as tf

import numpy as np
import os
import time

In [3]:
path_to_file = 'anekdots.txt'

In [6]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print('Length of text: {} characters'.format(len(text)))

Length of text: 511069 characters


In [7]:
print(text[:500])

1




Еду, никого не трогаю, тормозят любимые сотрудники ГАИ.

- Ваши документы... Петр Петрович, а чего глаза такие красные в выходные отдыхать надо...

- Да отдыхал как раз с друзьями в бане...

- Поди с девочками?

- Да нет, чисто мужская компания...

- А, значит с пивом, да с водочкой?

- Да нет, ничего не пили...

- Ну, пойдемте, подышим в трубочку

- Пойдемте...

Трубочка показывает что я абсолютно трезв. Возвращают документы...

- Хреново отдыхаете, Петр Петрович!





2




Приходит как-


In [8]:
text = text + text

In [9]:
# The unique characters in the file
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))

161 unique characters


In [10]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [11]:
text_as_int, text, len(text_as_int), len(text)

(array([18,  0,  0, ...,  0,  0,  0]),
 '1\n\n\n\n\nЕду, никого не трогаю, тормозят любимые сотрудники ГАИ.\n\n- Ваши документы... Петр Петрович, а чего глаза такие красные в выходные отдыхать надо...\n\n- Да отдыхал как раз с друзьями в бане...\n\n- Поди с девочками?\n\n- Да нет, чисто мужская компания...\n\n- А, значит с пивом, да с водочкой?\n\n- Да нет, ничего не пили...\n\n- Ну, пойдемте, подышим в трубочку\n\n- Пойдемте...\n\nТрубочка показывает что я абсолютно трезв. Возвращают документы...\n\n- Хреново отдыхаете, Петр Петрович!\n\n\n\n\n\n2\n\n\n\n\nПриходит как-то Рабинович к местечковому ребе.\n\n- Ребе, можно курить в субботу?\n\n- Конечно, нет!\n\n- Но вы же курите?!\n\n- Так я ни у кого и не спрашиваю!\n\n\n\n\n\n3\n\n\n\n\n- Просто интересно, как это на фотографиях вашего ателье все так непринужденно смеются?\n\n- Вы бы так не удивлялись, если бы знали, как выглядит наш фотограф!\n\n\n\n\n\n4\n\n\n\n\nВстречаются два мужика.\n\n- Где пропадал?\n\n- Да вот, детей возил в г

### train and target

In [12]:
# The maximum length sentence you want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

1










In [13]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

'1\n\n\n\n\nЕду, никого не трогаю, тормозят любимые сотрудники ГАИ.\n\n- Ваши документы... Петр Петрович, а ч'
'его глаза такие красные в выходные отдыхать надо...\n\n- Да отдыхал как раз с друзьями в бане...\n\n- Под'
'и с девочками?\n\n- Да нет, чисто мужская компания...\n\n- А, значит с пивом, да с водочкой?\n\n- Да нет, н'
'ичего не пили...\n\n- Ну, пойдемте, подышим в трубочку\n\n- Пойдемте...\n\nТрубочка показывает что я абсолю'
'тно трезв. Возвращают документы...\n\n- Хреново отдыхаете, Петр Петрович!\n\n\n\n\n\n2\n\n\n\n\nПриходит как-то Ра'


In [14]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

Print the first example input and target values:

In [15]:
for input_example, target_example in  dataset.take(1):
    print('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  '1\n\n\n\n\nЕду, никого не трогаю, тормозят любимые сотрудники ГАИ.\n\n- Ваши документы... Петр Петрович, а '
Target data: '\n\n\n\n\nЕду, никого не трогаю, тормозят любимые сотрудники ГАИ.\n\n- Ваши документы... Петр Петрович, а ч'


In [16]:
# Batch size
BATCH_SIZE = 64

BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [17]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [18]:
model = tf.keras.Sequential(
    [
        tf.keras.layers.Embedding(vocab_size, embedding_dim),
        tf.keras.layers.LSTM(rnn_units, return_sequences=True),
        tf.keras.layers.Dense(vocab_size)
    ]
)

In [19]:
class RNNgenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, batch_size):
        super(RNNgenerator, self).__init__()
        
        self.emb = tf.keras.layers.Embedding(vocab_size, embedding_dim)
                                 
        self.gru1 = tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            recurrent_initializer='glorot_uniform')
        self.gru2 = tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            recurrent_initializer='glorot_uniform')
                           
        self.fc = tf.keras.layers.Dense(vocab_size)

    def call(self, x):
        emb_x = self.emb(x)
        x1 = self.gru1(emb_x)
        x = x1
        for _ in range(3):
            x = self.gru2(x)
        #x = self.gru1(x)
        x = (x + x1)/2
        return self.fc(x)

model = RNNgenerator(vocab_size, embedding_dim, BATCH_SIZE)

In [19]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
                                 
        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

         tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        
        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
                                   
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [20]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim),
                                 
        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=False,
                            recurrent_initializer='glorot_uniform'),

        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=False,
                            recurrent_initializer='glorot_uniform'),

         tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=False,
                            recurrent_initializer='glorot_uniform'),
                                   
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [21]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [22]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 161) # (batch_size, sequence_length, vocab_size)


In [23]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, None, 256)         41216     
                                                                 
 gru_2 (GRU)                 (None, None, 1024)        3938304   
                                                                 
 gru_3 (GRU)                 (None, None, 1024)        6297600   
                                                                 
 gru_4 (GRU)                 (None, None, 1024)        6297600   
                                                                 
 dense_2 (Dense)             (None, None, 161)         165025    
                                                                 
Total params: 16,739,745
Trainable params: 16,739,745
Non-trainable params: 0
_________________________________________________________________


In [24]:
example_batch_predictions[0]

<tf.Tensor: shape=(100, 161), dtype=float32, numpy=
array([[ 4.6670364e-04,  1.5688903e-04,  2.3425500e-04, ...,
        -5.2156847e-04, -1.1942831e-03,  1.4466553e-03],
       [ 6.4101710e-05,  1.0167763e-03, -7.5313193e-04, ...,
        -9.7472512e-04, -1.1311946e-03,  2.0789634e-03],
       [-3.1671766e-04,  1.3615880e-03, -7.5378339e-04, ...,
         3.1903503e-05, -5.6495215e-04,  2.1539486e-03],
       ...,
       [-8.7118857e-03,  3.6721888e-03, -7.9486212e-03, ...,
         6.2111742e-03,  8.9628575e-04,  1.1419859e-03],
       [-1.0350323e-02,  5.9217727e-03, -1.0173468e-02, ...,
         7.1927952e-03,  8.0619694e-04, -6.8591908e-07],
       [-1.1482109e-02,  7.8486819e-03, -1.1839410e-02, ...,
         7.5051640e-03,  8.1093039e-04, -9.3727000e-04]], dtype=float32)>

In [25]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [26]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 'ть своя отдельная комната, и там она веселится вовсю: хочет- борщ варит, хочет- посуду моет. .\n\n\n\n\n\n'

Next Char Predictions: 
 "hчAь-У–_bЧwc\ufeffУBzУCЪn\n'_БЬб4.фМN$zа«Cgуп\ufeffбlBЪCЦаяё8„DЖщfип“#!Tсs\nН@8ЁMИи©dЪ0\nQK\nЁ0N;e)15%zЖnВ(кЗбkV5Ы"


## Train the model

In [27]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 161)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       5.08139


In [28]:
model.compile(optimizer='adam', loss=loss)

### Configure checkpoints

In [30]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_freq=88*5,
    save_weights_only=True)

### Execute the training

In [33]:
EPOCHS = 10

In [34]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
158/158 [==============================] - 941s 6s/step - loss: 2.2602
Epoch 2/10
158/158 [==============================] - 7010s 45s/step - loss: 1.8721
Epoch 3/10
130/158 [=======================>......] - ETA: 2:19 - loss: 1.6116

KeyboardInterrupt: 

In [35]:
history = model.fit(dataset, epochs=1, callbacks=[checkpoint_callback])

158/158 [==============================] - 805s 5s/step - loss: 1.4248


## Generate text

In [36]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_3'

In [37]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [38]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, None, 256)         41216     
                                                                 
 gru_5 (GRU)                 (None, None, 1024)        3938304   
                                                                 
 gru_6 (GRU)                 (None, None, 1024)        6297600   
                                                                 
 gru_7 (GRU)                 (None, None, 1024)        6297600   
                                                                 
 dense_3 (Dense)             (None, None, 161)         165025    
                                                                 
Total params: 16,739,745
Trainable params: 16,739,745
Non-trainable params: 0
_________________________________________________________________


In [71]:
def generate_text(model, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 100

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.
    temperature = 0.7

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [57]:
text_ = generate_text(model, start_string=u"Приходит полицейский в магазин ")
print(text_)

Приходит полицейский в магазин и ком ни -оронани Манае пи толов вона, -ой во, сь сонолетоноваватопит му: У“&олист налетущесту чк по


In [68]:
text_ = generate_text(model, start_string=u"Шерлок Холмс и Доктор Ватсон ")
print(text_)

Шерлок Холмс и Доктор Ватсон и пора пора пора пора пора пора пора пора пора пора пора пора пора пора пора пора пора пора пора пор


In [72]:
text_ = generate_text(model, start_string=u"Шерлок Холмс и Доктор Ватсон ")
print(text_)

Шерлок Холмс и Доктор Ватсон подетсопа е порсконет к па у де накобородлиск - бомета поро ности вадить хонаст поранет страманаегоб
